In [1]:
import open3d as o3d
import numpy as np
import sys
import joblib

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
sys.path.append('../sample')
from readdatas import *

In [4]:
path = 'D:\project\Point_Datas\Point Cloud Data\Corner.ply'
pcd =  ReadDatas(path)

In [5]:
gpf = GpfGroundExtractor()
gpf.set_data(pcd)
gpf._process_data()

In [9]:
class SVM(BaseAlgorithm):

    def __init__(self, data, path):
        self.data = data;
        self.ground = data.ground
        self.ground_o3d = self.np_to_o3d(self.ground)
        self.ground_nor = np.asarray(self.ground_o3d.normals)
        self.feature_vector = self.get_feature_vector()
        self.path = path

    def _process_data(self):
        clf = joblib.load(self.path)
        y_pred_all = clf.predict(self.feature_vector)
        index = np.array(range(len(pcd.ground)))
        index = index[y_pred_all]
        self.no_side = self.ground_o3d.select_by_index(index)
        self.data.paves = self.euclidean_cluster(self.no_side)   # 欧拉点数目大于150个的点都是avaible for pave. 大约6000个点


    def euclidean_cluster(self,cloud, tolerance=0.2):
        seed = self.seed_select(cloud)
        no_paves = self.find_no_paves(cloud)
        paves = np.asarray(cloud.points)
        no_paves = self.find_index(paves, no_paves)
        kdtree = o3d.geometry.KDTreeFlann(cloud)
        paves = np.array(seed)
        while len(seed) > 0 :
            seed_now = seed[0]
            k, idx, _ = kdtree.search_radius_vector_3d(cloud.points[seed_now], tolerance)
            if k == 1 :
                continue
            idx = np.array(idx)
            idx = idx[np.isin(idx, paves, invert=True)]
            idx = idx[np.isin(idx, no_paves, invert=True)]
            paves = np.append(paves, idx)
            seed = np.append(seed,idx)
            seed = np.delete(seed,[0])
        return paves

        

    def get_feature_vector(self):
        ground_tree = o3d.geometry.KDTreeFlann(self.ground_o3d)
        n = len(self.ground)
        feature_vector = np.zeros((n,2))
        for i in range(n):
            [num, idx, _] = ground_tree.search_radius_vector_3d(self.ground_o3d.points[i], 0.2)
            point_neighbour = self.ground[idx,:]
            point_neighbour_max = point_neighbour[:,2].max(axis=0)
            point_neighbour_min = point_neighbour[:,2].min(axis=0)
            point_neighbour_h = point_neighbour_max - point_neighbour_min
            point_neighbour_var = np.var(point_neighbour[:,2],axis=0)
            feature_vector[i,:] = [point_neighbour_h,point_neighbour_var]
        feature_vector = np.hstack((feature_vector,self.ground_nor.reshape(-1,3)))
        return feature_vector



In [10]:
path = 'SVM_model.pkl'

In [11]:
svm = SVM(pcd,path)

In [9]:

feature_vector = svm.feature_vector

In [12]:
svm._process_data()

In [11]:
paves = svm.no_side.select_by_index(pcd.paves)

In [ ]:
o3d.visualization.draw_geometries([paves])

In [ ]:
ground = pcd.ground
ground_o3d = gpf.np_to_o3d(pcd.ground)

In [ ]:
curvity = gpf.curvature_calculation(ground_o3d)

In [ ]:
ground_nor = np.asarray(ground_o3d.normals)

In [ ]:
ground_tree = o3d.geometry.KDTreeFlann(ground_o3d)
n = len(ground)
feature_vector = np.zeros((n,2))
for i in range(n):
    [num, idx, _] = ground_tree.search_radius_vector_3d(ground_o3d.points[i], 0.2)
    point_neighbour = ground[idx,:]
    point_neighbour_max = point_neighbour[:,2].max(axis=0)
    point_neighbour_min = point_neighbour[:,2].min(axis=0)
    point_neighbour_h = point_neighbour_max - point_neighbour_min
    point_neighbour_var = np.var(point_neighbour[:,2],axis=0)
    feature_vector[i,:] = [point_neighbour_h,point_neighbour_var]

In [ ]:
feature_vector = np.hstack((feature_vector,ground_nor.reshape(-1,3)))
# feature_vector = np.hstack((feature_vector,curvity.reshape(-1,1)))

In [ ]:
colors[:,:] = [1,0,0]
colors[pcd.no_paves] = [0,1,0]

In [ ]:
ground_o3d.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([ground_o3d])

In [ ]:
reg = ReGrowSegment(pcd)
reg._process_data()

In [ ]:
paves = ground_o3d.select_by_index(pcd.paves)
o3d.visualization.draw_geometries([paves])

In [ ]:
dps = DriPathSegment()
dps.set_data(pcd)
dps._process_data()

In [ ]:
y = np.full(len(ground),False,dtype=bool)
y[pcd.paves] = True

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
import joblib
clf = joblib.load('SVM_model.pkl')
# joblib.dump(clf, 'SVM_model.pkl')

In [ ]:
# 创建一个SVM模型，使用RBF核函数
clf = svm.SVC(kernel='rbf', C=1, gamma=0.5)

# 训练模型
clf.fit(feature_vector, y)

In [ ]:
y_pred = clf.predict(feature_vector[pcd.paves])
y_pred_all = clf.predict(feature_vector)

In [ ]:
accuracy = accuracy_score(y[pcd.paves], y_pred)

# 输出准确率
print("Accuracy: %0.2f" % accuracy)

accuracy_all = accuracy_score(y, y_pred_all)

# 输出准确率
print("Accuracy: %0.2f" % accuracy_all)

In [ ]:
index = np.array(range(len(pcd.ground)))
index = index[y_pred_all]
paves1 = ground_o3d.select_by_index(index)
o3d.visualization.draw_geometries([paves1])